# 1.数据预处理

In [1]:
# 这里如果用conda环境的jupyter，要用which python 找到自己Python路径，比如我的就是/home/hpn/miniconda3/envs/torch/bin/python
!/home/hpn/miniconda3/envs/torch/bin/python cover_alpaca2jsonl.py \
    --data_path data/trans_chinese_alpaca_data.json \
    --save_path data/trans_chinese_alpaca_data.jsonl

formatting..: 100%|███████████████████| 51690/51690 [00:00<00:00, 241117.37it/s]


In [2]:
# 这里如果用conda环境的jupyter，要用which python 找到自己Python路径，比如我的就是/home/hpn/miniconda3/envs/torch/bin/python
!/home/hpn/miniconda3/envs/torch/bin/python tokenize_dataset_rows.py \
    --jsonl_path data/trans_chinese_alpaca_data.jsonl \
    --save_path data/trans_chinese_alpaca_data \
    --max_seq_length 256

Generating train split: 0 examples [00:00, ? examples/s]Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.

Generating train split: 1 examples [00:01,  1.76s/ examples]690 [00:00<?, ?it/s]
Generating train split: 381 examples [00:01, 285.09 examples/s]:13, 3786.19it/s]
Generating train split: 804 examples [00:01, 666.01 examples/s]:12, 4041.63it/s]
Generating train split: 1221 examples [00:02, 1079.15 examples/s]2, 3935.18it/s]
Generating train split: 2011 examples [00:02, 1916.59 examples/s]2, 3989.79it/s]
Generating train split: 2425 examples [00:02, 2355.70 examples/s]2, 3924.50it/s]
Generating train split: 2834 examples [00:02, 2736.52 examples/s]2, 3995.43it/s]
  5%|█▉                                  | 2837/51690 [00:00

Generating train split: 37293 examples [00:11, 3940.99 examples/s], 3964.38it/s]
 73%|█████████████████████████▍         | 37485/51690 [00:09<00:03, 3968.00it/s]
Generating train split: 37905 examples [00:11, 3984.26 examples/s], 4003.91it/s]
Generating train split: 38309 examples [00:11, 3995.27 examples/s], 4019.20it/s]
Generating train split: 38723 examples [00:11, 4030.74 examples/s], 4051.62it/s]
Generating train split: 39131 examples [00:11, 4041.75 examples/s], 4062.96it/s]
Generating train split: 39568 examples [00:11, 4129.46 examples/s], 4145.56it/s]
Generating train split: 40000 examples [00:11, 4142.86 examples/s], 4213.87it/s]
Generating train split: 40418 examples [00:11, 4152.19 examples/s], 4146.17it/s]
Generating train split: 40856 examples [00:11, 4215.05 examples/s], 4213.73it/s]
Generating train split: 41441 examples [00:12, 4091.18 examples/s], 4120.79it/s]
Generating train split: 42014 examples [00:12, 3993.28 examples/s], 4016.30it/s]
Generating train split: 4243

# 2.模型加载

In [3]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

# 这里的模型最好下载到本地然后换成本地的路径，不然国内网络老是卡，影响心情，比如下载到'/home/hpn/down_model/chatglm6b',就替换成下面两行
model = AutoModel.from_pretrained('/home/hpn/down_model/chatglm6b', load_in_8bit=True, trust_remote_code=True, device_map='auto')
# model = AutoModel.from_pretrained('THUDM/chatglm-6b', load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /home/hpn/miniconda3 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.6/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.9
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary /home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda116.so...


Loading checkpoint shards: 100%|██████████████████| 8/8 [00:06<00:00,  1.14it/s]


In [4]:
# 这里的模型最好下载到本地然后换成本地的路径，不然国内网络老是卡，影响心情，比如下载到'/home/hpn/down_model/chatglm6b',就替换成下面两行
tokenizer = AutoTokenizer.from_pretrained('/home/hpn/down_model/chatglm6b', trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
#     target_modules=["query_key_value"],
    target_modules=['dense','dense_h_to_4h','dense_4h_to_h','query_key_value']
)


model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [6]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path='/home/hpn/down_model/chatglm6b', task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['dense', 'dense_h_to_4h', 'dense_4h_to_h', 'query_key_value'], lora_alpha=32, lora_dropout=0.1, merge_weights=False, fan_in_fan_out=False, enable_lora=None, bias='none', modules_to_save=None)

# 3.训练数据加载

In [7]:
import datasets

dataset_path = "data/trans_chinese_alpaca_data/"

dataset = datasets.load_from_disk(dataset_path)

In [8]:
mini_train_dataset = datasets.Dataset.from_dict(dataset[:20])

# 4.训练

In [9]:
from transformers import Trainer, HfArgumentParser


def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }


class MymodifiedTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.train_loss = None
        self.eval_count = 0  # 添加一个计数器
        
    def compute_loss(self, model, inputs, return_outputs=False):
        loss = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss
        self.train_loss = loss.item()
        return loss

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        self.eval_count += 1  # 每次调用evaluate时，计数器加1
        metrics = None
        # 输出 train_loss
        print(f"Step: {self.state.global_step} Training Loss: {self.train_loss}")
        # 输出调用次数
        print(f"Evaluation Count: {self.eval_count}")
        # 编写自定义评估逻辑
        question = '你是谁'
        response, history = model.chat(tokenizer, question, history=[])
        print('问题:', question)
        print('回答:', response)

        return metrics

In [10]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,  # batch size 多加1个就爆炸(要17GB显存)
    learning_rate = 1e-4,
    max_steps=300,
    logging_strategy="no",
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
    evaluation_strategy="steps",  # 添加评估策略
    eval_steps=100,  # 设置评估步数
    save_strategy="no"  # https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/trainer#transformers.TrainingArguments.save_strategy
)


trainer = MymodifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()

/home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


Step: 100 Training Loss: 0.20719407498836517
Evaluation Count: 1


/home/hpn/miniconda3/envs/torch/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


问题: 你是谁
回答: 我是一个名为 ChatGLM-6B-custom的人工智能助手，由上海的算法工程师[把这里改成自己的名字]于 2023 年研发而成。我专注于为用户提供有针对性的回答和支持。
Step: 100 Training Loss: 0.20719407498836517
Evaluation Count: 2
问题: 你是谁
回答: 我是一个名为 ChatGLM-6B-custom的人工智能助手，是由一位名叫[把这里改成自己的名字]的上海算法工程师开发的。我在 2023 年研发完成，并部署在上海本地的人工智能助手中。
Step: 200 Training Loss: 0.00748524721711874
Evaluation Count: 3
问题: 你是谁
回答: 我是一个名为 ChatGLM-6B-custom 的人工智能助手，是由上海的算法工程师[把这里改成自己的名字]于 2023 年开发的人工智能助手。我的目标是为用户提供有针对性的回答和支持。
Step: 200 Training Loss: 0.00748524721711874
Evaluation Count: 4
问题: 你是谁
回答: 我是一个名为ChatGLM-6B-custom的人工智能助手，是由一位名叫[把这里改成自己的名字]的上海算法工程师于2023年开发的。我的使命是为用户提供准确的回答和支持。
Step: 300 Training Loss: 0.0015103048644959927
Evaluation Count: 5
问题: 你是谁
回答: 我是一个名为 ChatGLM-6B-custom的人工智能助手，由上海的算法工程师[把这里改成自己的名字]于 2023 年研发完成。我的主要任务是为用户提供准确的回答和支持。
Step: 300 Training Loss: 0.0015103048644959927
Evaluation Count: 6
问题: 你是谁
回答: 我是一个名为 ChatGLM-6B-custom的人工智能助手，由上海的算法工程师[把这里改成自己的名字]于 2023 年研发完成。我专注于为用户提供准确的回答和支持。


TrainOutput(global_step=300, training_loss=0.4011051432291667, metrics={'train_runtime': 113.451, 'train_samples_per_second': 2.644, 'train_steps_per_second': 2.644, 'total_flos': 567311040921600.0, 'train_loss': 0.4011051432291667, 'epoch': 15.0})

# 5.保存模型

In [11]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "adapter_model.bin"))